<h1> This notebook shows how to convert a workflow instance into a QUBO </h1>

<div class="alert alert-block alert-info">
    <b> Create a workflow instance. </b>
</div>

In [1]:
from QHyper.problems.workflow_scheduling import Workflow, WorkflowSchedulingProblem

In [2]:
tasks_file = # todo add as a string the path to the tasks file e.g., workflows_data/workflows/3_tasks_3_machines_1_path.json
machines_file = # todo add as a string the path to the machines file e.g., workflows_data/machines/machines_for_3_tasks_3_machines_1_path.json
deadline = 20

In [3]:
workflow = Workflow(tasks_file, machines_file, deadline)
wsp = WorkflowSchedulingProblem(workflow)

In [4]:
wsp.workflow.time_matrix

,MachineA,MachineB,MachineC
task1,6.0,2.0,4.0
task2,3.0,1.0,2.0
task3,12.0,4.0,8.0


In [5]:
wsp.workflow.cost_matrix

,MachineA,MachineB,MachineC
task1,6.0,8.0,8.0
task2,3.0,4.0,4.0
task3,12.0,16.0,16.0


In [6]:
print(f"The objective function as a string is:\n {wsp.objective_function.polynomial}")

The objective function as a string is:
 6.0*x0 + 8.0*x1 + 8.0*x2 + 3.0*x3 + 4.0*x4 + 4.0*x5 + 12.0*x6 + 16.0*x7 + 16.0*x8


In [7]:
print(f"The objective function as a dict is:\n {wsp.objective_function.as_dict()}")

The objective function as a dict is:
 {('x0',): 6.0, ('x1',): 8.0, ('x2',): 8.0, ('x3',): 3.0, ('x4',): 4.0, ('x5',): 4.0, ('x6',): 12.0, ('x7',): 16.0, ('x8',): 16.0}


In [8]:
print("Constraints as dicts:")
for constraint in wsp.constraints:
    print(constraint.as_dict())

Constraints as dicts:
{('x0',): 1, ('x1',): 1, ('x2',): 1, (): -1}
{('x3',): 1, ('x4',): 1, ('x5',): 1, (): -1}
{('x6',): 1, ('x7',): 1, ('x8',): 1, (): -1}
{('s0',): 1, ('s1',): 2, ('s2',): 4, ('s3',): 6, ('x0',): 6.0, ('x1',): 2.0, ('x2',): 4.0, ('x3',): 3.0, ('x4',): 1.0, ('x5',): 2.0, ('x6',): 12.0, ('x7',): 4.0, ('x8',): 8.0, (): -20}


In [9]:
wsp.get_deadlines() # those are the fastest and the slowest execution times

(7.0, 21.0)

<div class="alert alert-block alert-info">
    <b> Convert the workflow to QUBO form. </b>
</div>

In [10]:
from QHyper.solvers.converter import Converter

In [11]:
converter = Converter()

The converter returns the QUBO as a dict, and numerical offset. We partially use [this](https://test-projecttemplate-dimod.readthedocs.io/en/latest/reference/bqm/generated/dimod.BinaryQuadraticModel.to_qubo.html) method.

In [12]:
wsp_qubo, offset = converter.to_qubo(wsp)

In [13]:
wsp_qubo

{('x1', 'x0'): 260.0,
 ('x2', 'x0'): 500.0,
 ('x2', 'x1'): 180.0,
 ('x3', 'x0'): 360.0,
 ('x3', 'x1'): 120.0,
 ('x3', 'x2'): 240.0,
 ('x4', 'x0'): 120.0,
 ('x4', 'x1'): 40.0,
 ('x4', 'x2'): 80.0,
 ('x4', 'x3'): 80.0,
 ('x5', 'x0'): 240.0,
 ('x5', 'x1'): 80.0,
 ('x5', 'x2'): 160.0,
 ('x5', 'x3'): 140.0,
 ('x5', 'x4'): 60.0,
 ('x6', 'x0'): 1440.0,
 ('x6', 'x1'): 480.0,
 ('x6', 'x2'): 960.0,
 ('x6', 'x3'): 720.0,
 ('x6', 'x4'): 240.0,
 ('x6', 'x5'): 480.0,
 ('x7', 'x0'): 480.0,
 ('x7', 'x1'): 160.0,
 ('x7', 'x2'): 320.0,
 ('x7', 'x3'): 240.0,
 ('x7', 'x4'): 80.0,
 ('x7', 'x5'): 160.0,
 ('x7', 'x6'): 980.0,
 ('x8', 'x0'): 960.0,
 ('x8', 'x1'): 320.0,
 ('x8', 'x2'): 640.0,
 ('x8', 'x3'): 480.0,
 ('x8', 'x4'): 160.0,
 ('x8', 'x5'): 320.0,
 ('x8', 'x6'): 1940.0,
 ('x8', 'x7'): 660.0,
 ('s0', 'x0'): 120.0,
 ('s0', 'x1'): 40.0,
 ('s0', 'x2'): 80.0,
 ('s0', 'x3'): 60.0,
 ('s0', 'x4'): 20.0,
 ('s0', 'x5'): 40.0,
 ('s0', 'x6'): 240.0,
 ('s0', 'x7'): 80.0,
 ('s0', 'x8'): 160.0,
 ('s1', 'x0'): 240.0

In [14]:
offset

4030.0

-----------------


<h3> This part shows how to use solvers from QHyper </h3>

<div class="alert alert-block alert-info">
    <b> Solve the Workflow Schedluling Problem with the CQM solver. </b>
</div>

In [15]:
from QHyper.solvers.CQM.cqm import CQM

In [16]:
cqm_problem = CQM(problem=wsp, time=5)

In [17]:
solution = cqm_problem.solve()
print(solution)

{'s0': 0.0, 's1': 0.0, 's2': 0.0, 's3': 0.0, 'x0': 1.0, 'x1': 0.0, 'x2': 0.0, 'x3': 0.0, 'x4': 0.0, 'x5': 1.0, 'x6': 1.0, 'x7': 0.0, 'x8': 0.0}


In [18]:
wsp.decode_solution(solution)

{'task1': 'MachineA', 'task2': 'MachineC', 'task3': 'MachineA'}

<div class="alert alert-block alert-info">
    <b> Solve the Workflow Schedluling Problem with the Gurobi solver. </b>
</div>

In [19]:
from QHyper.solvers.gurobi.gurobi import Gurobi

In [20]:
gurobi = Gurobi(problem=wsp)

In [21]:
solution_gurobi = gurobi.solve()

Set parameter Username
Academic license - for non-commercial use only - expires 2023-12-04
-1.0 + x0 + x1 + x2
-1.0 + x3 + x4 + x5
-1.0 + x6 + x7 + x8
-20.0 + s0 + 2.0 s1 + 4.0 s2 + 6.0 s3 + 6.0 x0 + 2.0 x1 + 4.0 x2 + 3.0 x3 + x4 + 2.0 x5 + 12.0 x6 + 4.0 x7 + 8.0 x8
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[rosetta2])

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4 rows, 13 columns and 22 nonzeros
Model fingerprint: 0xe016e177
Variable types: 0 continuous, 13 integer (13 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [3e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 28.0000000
Presolve removed 4 rows and 13 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processo

In [22]:
solution_gurobi

{'x0': 1.0,
 'x1': 0.0,
 'x2': 0.0,
 'x3': 0.0,
 'x4': 1.0,
 'x5': 0.0,
 'x6': 1.0,
 'x7': 0.0,
 'x8': 0.0,
 's0': 1.0,
 's1': 0.0,
 's2': 0.0,
 's3': 0.0}

In [23]:
wsp.decode_solution(solution_gurobi)

{'task1': 'MachineA', 'task2': 'MachineB', 'task3': 'MachineA'}